### General idea
This notebook shows the process from raw FBRef data to an exported csv file for future use:
* data scraping - using pandas' read_html, requests, and StringIO methods in a reusable function to customize league, season, and stat types
* data cleaning - cleaning up column names, dropping duplicates, and making data more readable
* feature engineering - converting data to per-90 (per game equivalent) basis, and creating new metrics derived from existing variables for better analysis
* data exporting - to a csv file for use in other projects

### Setting up basic libraries

* <b>numpy, pandas</b> - for data processing
* <b>requests, StringIO</b> - for data scraping
* <b>time</b> - delay/sleep between requests
* <b>timeit</b> - measuring speed/performance
* <b>warnings</b> - suppressing warnings (mainly performance warnings due to dataframe sizes)

In [1]:
import numpy as np
import pandas as pd
import time
import timeit
import warnings
import requests
from io import StringIO

### Setting up a dictionary of league IDs
* <b>leagues_url_dict</b> - mapping league inputs to IDs for url requests <br>
* <b>leagues_name_dict</b> - mapping league inputs to readable names for display in tables

In [2]:
leagues_url_dict = {
    "Big5": "Big-5-European-Leagues",
    "Championship": "10",
    "Serie-B": "18",
    "Major-League-Soccer": "22",
    "Eredivisie": "23",
    "Primeira-Liga": "32",
    "Belgian-Pro-League": "37"
}

leagues_name_dict = {
    "eng Premier League": "ENG Premier League",
    "de Bundesliga": "GER Bundesliga",
    "fr Ligue 1": "FRA Ligue 1",
    "es La Liga": "ESP La Liga",
    "it Serie A": "ITA Serie A",
    "Eredivisie": "NED Eredivisie",
    "Primeira-Liga": "POR Primeira Liga",
    "Belgian-Pro-League": "BEL Pro League",
    "Championship": "ENG2 Championship",
    "Serie-B": "ITA2 Serie B",
    "Major-League-Soccer": "USA Major League Soccer"
}

### Selecting leagues and types of stats to scrape
Every combination of items in <b>leagues</b> and <b>stats_reqd</b> will be scraped:<br>
* <b>leagues</b> contains the list of leagues to scrape
* <b>stats_reqd</b> contains the list of type of stats to scrape

In [3]:
leagues = ["Big5", "Eredivisie", "Primeira-Liga", "Belgian-Pro-League"]
stats_reqd = ["stats", "shooting", "passing", "passing_types", "gca", "defense", "possession", "misc"]

### Defining a reusable function to scrape data from FBREF
Function <b>clean</b> uses pandas' <b>read_html</b> function and takes in three inputs:<br>
* league - league to scrape (e.g. from list <b>leagues</b> in previous block)
* year - season (e.g. "2023-2024" used in the code)
* type - type of stat (e.g. from list <b>stats_reqd</b> in previous block)

What <b>clean</b> does:<br>
* Part A: Check variables <b>league</b>, <b>year</b> and <b>type</b> to generate url<br>
* Part B: Uses pandas' <b>read_html</b> with <b>requests</b> and <b>StringIO</b> to scrape the needed dataframe<br>
* Part C: Cleans up dataframe, specifically the column names with multi-indexes and extra spaces

In [4]:
def clean(league, year, reqd):
    # PART A
    # Check which league(s) to get needed league IDs for URL
    if league == "Big5":
        league2 = "Big5"
        league = leagues_url_dict[league2]
    else:
        league2 = leagues_url_dict[league]
    # Check type of table to get table_id, specific URL
    if reqd == "table":
        url = "https://fbref.com/en/comps/" + league2 + "/" + year + "/" + year + "-" + league + "-Stats"
        if league == "Big-5-European-Leagues":
            table_id = "big5_table"
        else:
            table_id = "results" + year + league2 + "1_overall"
    else:
        if reqd == "stats":
            table_id = "stats_standard"            
        elif reqd == "playingtime":
            table_id = "stats_playing_time"
        else:
            table_id = "stats_" + reqd
        url = "https://fbref.com/en/comps/" + league2 + "/" + year + "/" + reqd + "/players/" + year + "-" + league + "-Stats"
    # PART B
    # pd.read_html to get table using specific URL
    df = pd.read_html(StringIO(str(requests.get(url).text.replace('<!--','').replace('-->',''))), attrs={'id':table_id})[0]
    df.columns = [' '.join(col).strip() for col in df.columns]
    df = df.reset_index(drop=True)
    # PART C
    # Setting up column names
    new_columns = []
    for col in df.columns:
        if 'level_0' in col:
            new_col = col.split()[-1]  # takes the last name
        else:
            new_col = col
        new_columns.append(new_col)
    df.columns = new_columns
    df = df.fillna(0)
    if reqd == "table":
        df.columns = df.columns.str.replace(' ', '')
    else:
        df = df[df['Player'] != 'Player']
        df = df.drop(['Rk'], axis=1)
        if league2 != "Big5":
            df["Comp"] = league
    return(df)

### Scraping the data
Iterating through each combination of leagues in <b>leagues</b> and stats type in <b>stats_reqd</b>, appending each dataframe to create one big dataset

In [5]:
frames = []
for x in stats_reqd:
    df = pd.DataFrame()
    for y in leagues:
        import_start = timeit.default_timer()
        df = pd.concat([df, clean(y, "2023-2024", x)], axis=0, sort=False).reset_index(drop=True)
        time.sleep(6)
        import_stop = timeit.default_timer()
        print("Imported %s %s in %.2f seconds" % (y, x, import_stop - import_start))
    frames.append(df)
dataset = pd.concat(frames, axis=1, sort=False).reset_index()

Imported Big5 stats in 11.02 seconds
Imported Eredivisie stats in 6.99 seconds
Imported Primeira-Liga stats in 6.91 seconds
Imported Belgian-Pro-League stats in 6.89 seconds
Imported Big5 shooting in 9.61 seconds
Imported Eredivisie shooting in 6.91 seconds
Imported Primeira-Liga shooting in 8.83 seconds
Imported Belgian-Pro-League shooting in 6.72 seconds
Imported Big5 passing in 11.21 seconds
Imported Eredivisie passing in 6.95 seconds
Imported Primeira-Liga passing in 8.40 seconds
Imported Belgian-Pro-League passing in 7.14 seconds
Imported Big5 passing_types in 8.99 seconds
Imported Eredivisie passing_types in 6.92 seconds
Imported Primeira-Liga passing_types in 6.90 seconds
Imported Belgian-Pro-League passing_types in 6.76 seconds
Imported Big5 gca in 9.54 seconds
Imported Eredivisie gca in 6.78 seconds
Imported Primeira-Liga gca in 7.22 seconds
Imported Belgian-Pro-League gca in 6.85 seconds
Imported Big5 defense in 9.61 seconds
Imported Eredivisie defense in 7.12 seconds
Importe

### Cleaning the dataset
* Dropping all duplicated columns
* Filling all NaN values with 0
* Cleaning the <b>Nation</b> column to three-letter codes
* Cleaning the <b>Comp</b> column to readable league names, reordering to left side of dataframe
* Dropping irrelevant columns: <b>90s</b> redundant with Minutes Played, <b>Matches</b> contains text/link only
* Converting all columns after player identifiers (categorical) to <b>numeric</b>
* Converting all non-percentage, non-"per-90" basis stats to <b>per-90</b> basis

In [6]:
dataset = dataset.loc[:,~dataset.columns.duplicated()].copy()
dataset['Nation'] = dataset['Nation'].str.split(' ').str.get(1)
dataset["Comp"] = dataset["Comp"].map(leagues_name_dict)
dataset.drop(dataset.filter(regex='90').columns, axis=1, inplace=True)
dataset.drop(dataset.filter(regex='Matches').columns, axis=1, inplace=True)
col = dataset.pop('Comp')
dataset.insert(5, col.name, col)
dataset.iloc[:,8:] = dataset.iloc[:,8:].apply(pd.to_numeric, errors='coerce', axis=1)
for col in dataset.columns[11:]:
    if ('90' in col) or ('%' in col):
        pass
    else:
        dataset[col] = dataset[col]/(dataset['Playing Time Min']/90)
dataset = dataset.fillna(0)
warnings.simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_26304\812199887.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset = dataset.fillna(0)


### Creating new variables
* <b>Success rate metrics:</b> Tackles Tkl%, Foul Prev%
* <b>Aggregated metrics:</b> Deep DAs and High DAs (Defensive Actions)
* <b>%Touches:</b> proportion of touches by specific zone (Def Pen, Def 3rd, Mid 3rd, Att 3rd, Att Pen)
* <b>%Pass Att:</b> proportion of attempted passes by pass length (Short, Medium, Long)
* <b>xAG/SCA:</b> like xG/Shot, but for chance creation

In [7]:
dataset["Tackles Tkl%"] = np.where(dataset["Tackles Tkl"]!=0, (100*dataset["Tackles TklW"].astype(float)/dataset["Tackles Tkl"].astype(float)), 0)
dataset["Foul Prev%"] = np.where(dataset["Tackles Tkl"]!=0, (100*1-(dataset["Performance Fls"].astype(float)/dataset["Tackles Tkl"].astype(float))), 0)
dataset["Touches %Def Pen"] = np.where(dataset["Touches Touches"]!=0, (100*dataset["Touches Def Pen"].astype(float)/dataset["Touches Touches"].astype(float)), 0)
dataset["Touches %Def 3rd"] = np.where(dataset["Touches Touches"]!=0, (100*dataset["Touches Def 3rd"].astype(float)/dataset["Touches Touches"].astype(float)), 0)
dataset["Touches %Mid 3rd"] = np.where(dataset["Touches Touches"]!=0, (100*dataset["Touches Mid 3rd"].astype(float)/dataset["Touches Touches"].astype(float)), 0)
dataset["Touches %Att 3rd"] = np.where(dataset["Touches Touches"]!=0, (100*dataset["Touches Att 3rd"].astype(float)/dataset["Touches Touches"].astype(float)), 0)
dataset["Touches %Att Pen"] = np.where(dataset["Touches Touches"]!=0, (100*dataset["Touches Att Pen"].astype(float)/dataset["Touches Touches"].astype(float)), 0)
dataset["%Short Att"] = np.where(dataset["Total Att"]!=0, (100*dataset["Short Att"].astype(float)/dataset["Total Att"].astype(float)), 0)
dataset["%Medium Att"] = np.where(dataset["Total Att"]!=0, (100*dataset["Medium Att"].astype(float)/dataset["Total Att"].astype(float)), 0)
dataset["%Long Att"] = np.where(dataset["Total Att"]!=0, (100*dataset["Long Att"].astype(float)/dataset["Total Att"].astype(float)), 0)
dataset["xAG/SCA"] = np.where(dataset["SCA SCA"]!=0, (100*dataset["xAG"].astype(float)/dataset["SCA SCA"].astype(float)), 0)
dataset["Deep DAs"] = dataset["Blocks Sh"] + dataset["Clr"] + dataset["Tackles Def 3rd"]
dataset["High DAs"] = dataset["Tackles Mid 3rd"] + dataset["Tackles Att 3rd"] + dataset["Int"]

### Exporting dataset
Exporting the dataset to a CSV file

In [8]:
dataset.to_csv("2324_top_8_raw.csv", encoding="utf-8", index=False)